In [0]:
import pandas as pd

user_emp_paydates_df_pd = pd.read_parquet("/Volumes/main_prod/datascience_scratchpad/jatin/trajcl_exp/nyc/hyp_2/user_emp_paydates_pck_df.parquet")

In [0]:
display(user_emp_paydates_df_pd.head())

In [0]:
gt_label_0_list_file = "/Workspace/Users/jatin.agrawal@earnin.com/TrajCL/data/gt_label_0_list.pkl"
import pickle

gt_label_0_list = pickle.load(open(gt_label_0_list_file, "rb"))

In [0]:
gt_label_0_list

In [0]:
import bisect
def get_paydates_range(user_id, emp, partition_id, start_date,end_date):
    df = user_emp_paydates_df_pd[(user_emp_paydates_df_pd["userid"] == user_id) & (user_emp_paydates_df_pd['employername']==emp)]
    partitions  = df['partitions'].values[0]
    partition = partitions[partition_id]
    paydates = partition['paydates']
    paydate_idx = bisect.bisect_right(paydates, end_date)
    if paydate_idx == len(paydates):
        return paydates[-2], paydates[-1]
    elif paydate_idx == 0:
        return start_date, paydates[0]
    else:
        return paydates[paydate_idx-1], paydates[paydate_idx]

In [0]:
gt_label_0_list[3]

In [0]:
display(user_emp_paydates_df_pd[(user_emp_paydates_df_pd["userid"] == gt_label_0_list[1]['key'][0]) & (user_emp_paydates_df_pd['employername']==gt_label_0_list[1]['key'][1])])

In [0]:

get_paydates_range(gt_label_0_list[0]['key'][0], gt_label_0_list[0]['key'][1], gt_label_0_list[0]['key'][2], gt_label_0_list[0]['traj_dates'][-1])

In [0]:
%sql
select * from main_prod.paycheckprediction.receivedpaycheck where userid  = "6048564"

In [0]:
from tqdm import tqdm
output = []


for i in tqdm(range(len(gt_label_0_list))):
    user_id = gt_label_0_list[i]['key'][0]
    emp = gt_label_0_list[i]['key'][1]
    partition_id = gt_label_0_list[i]['key'][2]
    start_date = gt_label_0_list[i]['traj_dates'][0]
    end_date = gt_label_0_list[i]['traj_dates'][-1]
    start_date, end_date = get_paydates_range(user_id, emp, partition_id, start_date, end_date)
    #increase end date by 2 days
    end_date = end_date + pd.DateOffset(days=2)
    query = """
    select amount, availableonest from main_prod.paycheckprediction.receivedpaycheck where userid = {} and availableonest between '{}' and '{}' 
    """.format(user_id, start_date, end_date)
    df = spark.sql(query)
    amount = 0
    if df.count()>0:
        amount = df.select("amount").collect()[0]['amount']
    output.append(amount)    

In [0]:
final_output = []

for i in range(len(gt_label_0_list)):
    final_output.append((gt_label_0_list[i]['key'], output[i]))

pickle.dump(final_output, open("/Workspace/Users/jatin.agrawal@earnin.com/TrajCL/data/gt_label_0_list_with_amt.pkl", "wb"))

In [0]:
non_0_count = 0
for amount in output:
    if amount>0:
        non_0_count+=1

print(non_0_count), print(len(output))

In [0]:
%sql

SELECT * FROM main_prod.ml_data.income_type_v2 WHERE userid = '34299'

In [0]:
%sql
select * from main_prod.earnings_analysis.fact_user_earnings_daily where userid = "3463200"

In [0]:
%sql
select * from main_prod.analysis.derived_userstate_bankconnection where userid = "6269644"

In [0]:
from tqdm import tqdm
output = []


for i in tqdm(range(len(gt_label_0_list))):
    user_id = gt_label_0_list[i]['key'][0]
    emp = gt_label_0_list[i]['key'][1]
    partition_id = gt_label_0_list[i]['key'][2]
    start_date = gt_label_0_list[i]['traj_dates'][0]
    end_date = gt_label_0_list[i]['traj_dates'][-1]
    start_date, end_date = get_paydates_range(user_id, emp, partition_id, start_date, end_date)
    query = """
    select connectionstatus from main_prod.analysis.derived_userstate_bankconnection where userid = {} and date_ between '{}' and '{}'
    """.format(user_id, start_date, end_date)
    df = spark.sql(query)
    df_pd = df.toPandas()
    connection_status_list = df_pd['connectionstatus'].values
    connection = "active"
    for status in connection_status_list:
        if status!='Healthy':
            connection = status
            break
    output.append(connection)

In [0]:
count_inactive = 0
inactive_dict = {}
for i,connection in enumerate(output):
    if connection!='active':
        count_inactive+=1
        inactive_dict[gt_label_0_list[i]['key'][0]] = connection
print(count_inactive), print(len(output))

In [0]:
inactive_dict[165431]

In [0]:
%sql
select * from main_prod.bankfeed.userprovidedbankconnection where userid = "6269644"

In [0]:
from tqdm import tqdm
import datetime
output = []


for i in tqdm(range(len(gt_label_0_list))):
    user_id = gt_label_0_list[i]['key'][0]
    emp = gt_label_0_list[i]['key'][1]
    partition_id = gt_label_0_list[i]['key'][2]
    start_date = gt_label_0_list[i]['traj_dates'][0]
    end_date = gt_label_0_list[i]['traj_dates'][-1]
    start_date, end_date = get_paydates_range(user_id, emp, partition_id, start_date, end_date)
    query = """
    select * from main_prod.bankfeed.userprovidedbankconnection where userid = "{}" and isactive = true and connectionstate = 1000
    """.format(user_id)
    df = spark.sql(query)
    df_pd = df.toPandas()
    connection = "inactive"
    last_healty_at_list = df_pd['lasthealthyat'].values
    created_on_list = df_pd['createdon'].values
    
    for healthy_date, created_on in zip(last_healty_at_list, created_on_list):
        created_on_date = datetime.datetime.strptime(created_on, "%Y-%m-%d %H:%M:%S").date()
        if created_on_date < start_date:
            connection = "active"
    output.append(connection)

In [0]:
display(df_pd)

In [0]:
count_inactive = 0
inactive_dict = {}
for i,connection in enumerate(output):
    if connection=='inactive':
        count_inactive+=1
        inactive_dict[gt_label_0_list[i]['key']] = "inactive"
print(count_inactive), print(len(output))

In [0]:
pickle.dump(inactive_dict, open("/Workspace/Users/jatin.agrawal@earnin.com/TrajCL/data/inactive_dict.pkl", "wb"))